# Crunch raw CMIP6 data

Here we calculate the country-means.

In [1]:
import netcdf_scm
import regionmask

In [2]:
netcdf_scm.__version__

'2.1.0'

In [3]:
!find /data/cmip6/CMIP6/ScenarioMIP -mindepth 2 -maxdepth 2 -type d -exec sh -c 'x={};echo $(basename ${x})' \; | sort

ACCESS-CM2
ACCESS-ESM1-5
AWI-CM-1-1-MR
BCC-CSM2-MR
CAMS-CSM1-0
CanESM5
CanESM5-CanOE
CAS-ESM2-0
CESM2
CESM2-WACCM
CIESM
CMCC-CM2-SR5
CMCC-ESM2
CNRM-CM6-1
CNRM-CM6-1-HR
CNRM-ESM2-1
E3SM-1-1
EC-Earth3
EC-Earth3-AerChem
EC-Earth3-CC
EC-Earth3-Veg
EC-Earth3-Veg-LR
FGOALS-f3-L
FGOALS-g3
FIO-ESM-2-0
GFDL-CM4
GFDL-ESM4
GISS-E2-1-G
HadGEM3-GC31-LL
HadGEM3-GC31-MM
IITM-ESM
INM-CM4-8
INM-CM5-0
IPSL-CM6A-LR
KACE-1-0-G
KIOST-ESM
MCM-UA-1-0
MIROC6
MIROC-ES2L
MPI-ESM-1-2-HAM
MPI-ESM1-2-HR
MPI-ESM1-2-HR
MPI-ESM1-2-LR
MRI-ESM2-0
NESM3
NorESM2-LM
NorESM2-MM
TaiESM1
UKESM1-0-LL
UKESM1-0-LL


In [4]:
CRUNCH_DIR = "./country-crunch"

In [5]:
!mkdir -p {CRUNCH_DIR}

In [6]:
too_small_regions = {
    "World|Natural Earth 50m|San Marino",
    "World|Natural Earth 50m|Turks and Caicos Is.",
    "World|Natural Earth 50m|Indian Ocean Ter.",
    "World|Natural Earth 50m|Falkland Is.",
    "World|Natural Earth 50m|Israel",
    "World|Natural Earth 50m|Kiribati",
    "World|Natural Earth 50m|Jamaica",
    "World|Natural Earth 50m|Fr. S. Antarctic Lands",
    "World|Natural Earth 50m|Vanuatu",
    "World|Natural Earth 50m|N. Mariana Is.",
    "World|Natural Earth 50m|Palau",
    "World|Natural Earth 50m|Liechtenstein",
    "World|Natural Earth 50m|St-Barthélemy",
    "World|Natural Earth 50m|Monaco",
    "World|Natural Earth 50m|Wallis and Futuna Is.",
    "World|Natural Earth 50m|Puerto Rico",
    "World|Natural Earth 50m|Bermuda",
    "World|Natural Earth 50m|Dominica",
    "World|Natural Earth 50m|Nauru",
    "World|Natural Earth 50m|Isle of Man",
    "World|Natural Earth 50m|Rwanda",
    "World|Natural Earth 50m|Grenada",
    "World|Natural Earth 50m|Seychelles",
    "World|Natural Earth 50m|S. Geo. and the Is.",
    "World|Natural Earth 50m|Cayman Is.",
    "World|Natural Earth 50m|Sint Maarten",
    "World|Natural Earth 50m|Maldives",
    "World|Natural Earth 50m|Norfolk Island",
    "World|Natural Earth 50m|St-Martin",
    "World|Natural Earth 50m|Bahamas",
    "World|Natural Earth 50m|British Virgin Is.",
    "World|Natural Earth 50m|Comoros",
    "World|Natural Earth 50m|St. Kitts and Nevis",
    "World|Natural Earth 50m|Barbados",
    "World|Natural Earth 50m|Palestine",
    "World|Natural Earth 50m|American Samoa",
    "World|Natural Earth 50m|St. Vin. and Gren.",
    "World|Natural Earth 50m|Brunei",
    "World|Natural Earth 50m|Curaçao",
    "World|Natural Earth 50m|Guam",
    "World|Natural Earth 50m|St. Pierre and Miquelon",
    "World|Natural Earth 50m|Jersey",
    "World|Natural Earth 50m|Macao",
    "World|Natural Earth 50m|Saint Lucia",
    "World|Natural Earth 50m|Cook Is.",
    "World|Natural Earth 50m|Fr. Polynesia",
    "World|Natural Earth 50m|Faeroe Is.",
    "World|Natural Earth 50m|Luxembourg",
    "World|Natural Earth 50m|Saint Helena",
    "World|Natural Earth 50m|Åland",
    "World|Natural Earth 50m|Niue",
    "World|Natural Earth 50m|Pitcairn Is.",
    "World|Natural Earth 50m|Br. Indian Ocean Ter.",
    "World|Natural Earth 50m|Vatican",
    "World|Natural Earth 50m|Heard I. and McDonald Is.",
    "World|Natural Earth 50m|Albania",
    "World|Natural Earth 50m|Malta",
    "World|Natural Earth 50m|N. Cyprus",
    "World|Natural Earth 50m|Ashmore and Cartier Is.",
    "World|Natural Earth 50m|Trinidad and Tobago",
    "World|Natural Earth 50m|Marshall Is.",
    "World|Natural Earth 50m|Belize",
    "World|Natural Earth 50m|U.S. Virgin Is.",
    "World|Natural Earth 50m|Antigua and Barb.",
    "World|Natural Earth 50m|Kuwait",
    "World|Natural Earth 50m|Siachen Glacier",
    "World|Natural Earth 50m|Andorra",
    "World|Natural Earth 50m|Singapore",
    "World|Natural Earth 50m|Micronesia",
    "World|Natural Earth 50m|Bahrain",
    "World|Natural Earth 50m|Togo",
    "World|Natural Earth 50m|Hong Kong",
    "World|Natural Earth 50m|Anguilla",
    "World|Natural Earth 50m|Mauritius",
    "World|Natural Earth 50m|Aruba",
    "World|Natural Earth 50m|São Tomé and Principe",
    "World|Natural Earth 50m|Solomon Is.",
    "World|Natural Earth 50m|Qatar",
    "World|Natural Earth 50m|Tonga",
    "World|Natural Earth 50m|Guernsey",
    "World|Natural Earth 50m|Montserrat",
    "World|Natural Earth 50m|Haiti",
}

In [7]:
regions = (
    set(
        [
            "World|Natural Earth 50m|{}".format(c)
            for c in regionmask.defined_regions.natural_earth.countries_50.names
        ]
        + ["World"]
    )
    - too_small_regions
)
display(len(regions))
regions = ",".join(regions)
# regions

160

In [8]:
!netcdf-scm crunch \
    /data/cmip6/CMIP6 \
    {CRUNCH_DIR} \
    "Zebedee Nicholls <zebedee.nicholls@climate-energy-college.org>" \
    --drs "CMIP6Output" \
    --regexp ".*(CMIP|ScenarioMIP).*(/ssp|/historical/|/piControl/).*(r1i1p1f1|r2i1p1f1|r3i1p1f1|r1i1p1f2|r1i1p2f1|r1i1p1f3|r4i1p1f1|r10i1p1f1|r11i1p1f1).*Amon.*/tas/.*" \
    --regions "World|Natural Earth 50m|Angola,World|Natural Earth 50m|Bulgaria,World|Natural Earth 50m|Mongolia,World|Natural Earth 50m|Lebanon,World|Natural Earth 50m|Turkey,World|Natural Earth 50m|Burundi,World|Natural Earth 50m|Mozambique,World|Natural Earth 50m|Uzbekistan,World|Natural Earth 50m|Zimbabwe,World|Natural Earth 50m|Belarus,World|Natural Earth 50m|Bolivia,World|Natural Earth 50m|Philippines,World|Natural Earth 50m|China,World|Natural Earth 50m|Argentina,World|Natural Earth 50m|Paraguay,World|Natural Earth 50m|Ireland,World|Natural Earth 50m|Switzerland,World|Natural Earth 50m|Croatia,World|Natural Earth 50m|Cameroon,World,World|Natural Earth 50m|Romania,World|Natural Earth 50m|United States of America,World|Natural Earth 50m|Kenya,World|Natural Earth 50m|Tanzania,World|Natural Earth 50m|Namibia,World|Natural Earth 50m|Georgia,World|Natural Earth 50m|Burkina Faso,World|Natural Earth 50m|Oman,World|Natural Earth 50m|Egypt,World|Natural Earth 50m|Costa Rica,World|Natural Earth 50m|Panama,World|Natural Earth 50m|Uruguay,World|Natural Earth 50m|Nepal,World|Natural Earth 50m|Suriname,World|Natural Earth 50m|Saudi Arabia,World|Natural Earth 50m|Lithuania,World|Natural Earth 50m|Macedonia,World|Natural Earth 50m|Dominican Rep.,World|Natural Earth 50m|Dem. Rep. Congo,World|Natural Earth 50m|S. Sudan,World|Natural Earth 50m|Kyrgyzstan,World|Natural Earth 50m|Niger,World|Natural Earth 50m|Eritrea,World|Natural Earth 50m|Djibouti,World|Natural Earth 50m|France,World|Natural Earth 50m|Greenland,World|Natural Earth 50m|Spain,World|Natural Earth 50m|Montenegro,World|Natural Earth 50m|Venezuela,World|Natural Earth 50m|Ethiopia,World|Natural Earth 50m|Iceland,World|Natural Earth 50m|Finland,World|Natural Earth 50m|Libya,World|Natural Earth 50m|Uganda,World|Natural Earth 50m|Algeria,World|Natural Earth 50m|Malaysia,World|Natural Earth 50m|United Arab Emirates,World|Natural Earth 50m|Sweden,World|Natural Earth 50m|Malawi,World|Natural Earth 50m|Iran,World|Natural Earth 50m|Indonesia,World|Natural Earth 50m|Cabo Verde,World|Natural Earth 50m|Slovenia,World|Natural Earth 50m|Guinea-Bissau,World|Natural Earth 50m|Armenia,World|Natural Earth 50m|Kosovo,World|Natural Earth 50m|Laos,World|Natural Earth 50m|South Africa,World|Natural Earth 50m|Sudan,World|Natural Earth 50m|Thailand,World|Natural Earth 50m|Tunisia,World|Natural Earth 50m|India,World|Natural Earth 50m|Estonia,World|Natural Earth 50m|Vietnam,World|Natural Earth 50m|Russia,World|Natural Earth 50m|Madagascar,World|Natural Earth 50m|Syria,World|Natural Earth 50m|Norway,World|Natural Earth 50m|W. Sahara,World|Natural Earth 50m|Mexico,World|Natural Earth 50m|Greece,World|Natural Earth 50m|Côte d'Ivoire,World|Natural Earth 50m|Germany,World|Natural Earth 50m|Japan,World|Natural Earth 50m|Somaliland,World|Natural Earth 50m|Italy,World|Natural Earth 50m|Australia,World|Natural Earth 50m|Zambia,World|Natural Earth 50m|Honduras,World|Natural Earth 50m|Jordan,World|Natural Earth 50m|Cyprus,World|Natural Earth 50m|Guinea,World|Natural Earth 50m|Antarctica,World|Natural Earth 50m|Tajikistan,World|Natural Earth 50m|Nigeria,World|Natural Earth 50m|Senegal,World|Natural Earth 50m|Sri Lanka,World|Natural Earth 50m|Congo,World|Natural Earth 50m|North Korea,World|Natural Earth 50m|Austria,World|Natural Earth 50m|Chile,World|Natural Earth 50m|Netherlands,World|Natural Earth 50m|Taiwan,World|Natural Earth 50m|Samoa,World|Natural Earth 50m|Denmark,World|Natural Earth 50m|Portugal,World|Natural Earth 50m|Bosnia and Herz.,World|Natural Earth 50m|Chad,World|Natural Earth 50m|eSwatini,World|Natural Earth 50m|Benin,World|Natural Earth 50m|Timor-Leste,World|Natural Earth 50m|Turkmenistan,World|Natural Earth 50m|Slovakia,World|Natural Earth 50m|Peru,World|Natural Earth 50m|Ecuador,World|Natural Earth 50m|Ukraine,World|Natural Earth 50m|Fiji,World|Natural Earth 50m|United Kingdom,World|Natural Earth 50m|New Zealand,World|Natural Earth 50m|Hungary,World|Natural Earth 50m|Gambia,World|Natural Earth 50m|New Caledonia,World|Natural Earth 50m|Ghana,World|Natural Earth 50m|Serbia,World|Natural Earth 50m|Central African Rep.,World|Natural Earth 50m|Nicaragua,World|Natural Earth 50m|Canada,World|Natural Earth 50m|Lesotho,World|Natural Earth 50m|Colombia,World|Natural Earth 50m|South Korea,World|Natural Earth 50m|Myanmar,World|Natural Earth 50m|Guatemala,World|Natural Earth 50m|Cuba,World|Natural Earth 50m|Yemen,World|Natural Earth 50m|Mali,World|Natural Earth 50m|Papua New Guinea,World|Natural Earth 50m|Somalia,World|Natural Earth 50m|Czechia,World|Natural Earth 50m|Moldova,World|Natural Earth 50m|Morocco,World|Natural Earth 50m|Poland,World|Natural Earth 50m|Bhutan,World|Natural Earth 50m|Pakistan,World|Natural Earth 50m|Sierra Leone,World|Natural Earth 50m|Guyana,World|Natural Earth 50m|Afghanistan,World|Natural Earth 50m|Liberia,World|Natural Earth 50m|Kazakhstan,World|Natural Earth 50m|Eq. Guinea,World|Natural Earth 50m|Brazil,World|Natural Earth 50m|El Salvador,World|Natural Earth 50m|Mauritania,World|Natural Earth 50m|Gabon,World|Natural Earth 50m|Botswana,World|Natural Earth 50m|Bangladesh,World|Natural Earth 50m|Cambodia,World|Natural Earth 50m|Azerbaijan,World|Natural Earth 50m|Iraq,World|Natural Earth 50m|Belgium,World|Natural Earth 50m|Latvia" \
    --small-number-workers 60 \
    --small-threshold 100 \
    --medium-number-workers 20 \
    --medium-threshold 400 \
    --force-lazy-threshold 1000 \
    > crunch-log.txt 2>&1

In [9]:
!find /data/cmip6/CMIP6/CMIP -name 'KIOST-ESM' -type d

/data/cmip6/CMIP6/CMIP/KIOST/KIOST-ESM


In [10]:
!ls /data/cmip6/CMIP6/ScenarioMIP/NCAR/CESM2

ssp126	ssp245	ssp370	ssp585


In [11]:
!find /data/cmip6/CMIP6/CMIP/KIOST/KIOST-ESM -type f -name 'tas_Amon*'

/data/cmip6/CMIP6/CMIP/KIOST/KIOST-ESM/piControl/r1i1p1f1/Amon/tas/gr1/v20191106/tas_Amon_KIOST-ESM_piControl_r1i1p1f1_gr1_268901-318812.nc
/data/cmip6/CMIP6/CMIP/KIOST/KIOST-ESM/abrupt-4xCO2/r1i1p1f1/Amon/tas/gr1/v20191106/tas_Amon_KIOST-ESM_abrupt-4xCO2_r1i1p1f1_gr1_185001-200012.nc
/data/cmip6/CMIP6/CMIP/KIOST/KIOST-ESM/historical/r1i1p1f1/Amon/tas/gr1/v20191106/tas_Amon_KIOST-ESM_historical_r1i1p1f1_gr1_185001-201412.nc


In [12]:
!find /data/cmip6/CMIP6/ScenarioMIP -name 'KIOST-ESM' -type d

/data/cmip6/CMIP6/ScenarioMIP/KIOST/KIOST-ESM


In [13]:
!find /data/cmip6/CMIP6/ScenarioMIP/KIOST/KIOST-ESM -type f -name 'tas_Amon*_ssp245_*'

/data/cmip6/CMIP6/ScenarioMIP/KIOST/KIOST-ESM/ssp245/r1i1p1f1/Amon/tas/gr1/v20191106/tas_Amon_KIOST-ESM_ssp245_r1i1p1f1_gr1_201501-210012.nc


In [14]:
!find /data/cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM -type f -name '*_ssp245_*'

/data/cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/Amon/tas/gn/v20200915/tas_Amon_IITM-ESM_ssp245_r1i1p1f1_gn_204501-205412.nc
/data/cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/Amon/tas/gn/v20200915/tas_Amon_IITM-ESM_ssp245_r1i1p1f1_gn_201501-202412.nc
/data/cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/Amon/tas/gn/v20200915/tas_Amon_IITM-ESM_ssp245_r1i1p1f1_gn_202501-203412.nc
/data/cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/Amon/tas/gn/v20200915/tas_Amon_IITM-ESM_ssp245_r1i1p1f1_gn_205501-206412.nc
/data/cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/Amon/tas/gn/v20200915/tas_Amon_IITM-ESM_ssp245_r1i1p1f1_gn_206501-207412.nc
/data/cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/Amon/tas/gn/v20200915/tas_Amon_IITM-ESM_ssp245_r1i1p1f1_gn_203501-204412.nc
/data/cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/Amon/tas/gn/v20200915/tas_Amon_IITM-ESM_ssp245_r1i1p1f1_gn_207501-208412.nc
/data/cmip6/C

In [15]:
!find {CRUNCH_DIR} -name '*_ssp245_*' -type f  #| wc -l

./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp245/r1i1p1f1/Amon/tas/gn/v20190314/netcdf-scm_tas_Amon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.nc
./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/Amon/tas/gn/v20200915/netcdf-scm_tas_Amon_IITM-ESM_ssp245_r1i1p1f1_gn_201501-209912.nc
./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp245/r2i1p1f1/Amon/tas/gr/v20201110/netcdf-scm_tas_Amon_E3SM-1-1_ssp245_r2i1p1f1_gr_202001-202412.nc
./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp245/r1i1p1f1/Amon/tas/gr/v20201109/netcdf-scm_tas_Amon_E3SM-1-1_ssp245_r1i1p1f1_gr_202001-202412.nc
./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp245/r10i1p1f1/Amon/tas/gr/v20201110/netcdf-scm_tas_Amon_E3SM-1-1_ssp245_r10i1p1f1_gr_202001-202412.nc
./country-crunch/netcdf-scm-crunched/CMIP6/ScenarioMIP/E3SM-Project/E3SM-1-1/ssp245/r4i1p1f1/Amon/tas/gr